In [1]:
import pandas as pd, matplotlib.pyplot as plt, os, sys, seaborn as sns
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [3]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
data = pd.read_csv('july/first_week/feat_ext64_mpu1.csv')
data.dtypes.value_counts()
print(data.shape)
Class = pd.read_csv('july/first_week/Ymotion_resize64_fe_mpu1.csv')
print(Class.shape)

(202, 114)
(202, 1)


In [5]:
Class.value_counts()

Class
5.0      44
3.0      42
1.0      39
4.0      39
2.0      38
Name: count, dtype: int64

In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
#y = pd.get_dummies(Class, columns=['Class'])
y = Class
X = data

In [11]:
# Convert labels to integers
label_mapping = {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4}
y['Class'] = y['Class'].map(label_mapping).astype(int)
print(y)

     Class
0        0
1        0
2        0
3        0
4        0
..     ...
197      4
198      4
199      4
200      4
201      4

[202 rows x 1 columns]


In [7]:
# Split the data into training and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing=None, seed=42)
clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric='mlogloss',
            eval_set=[(X_test, y_test)])

[0]	validation_0-mlogloss:1.20729
[1]	validation_0-mlogloss:0.98024
[2]	validation_0-mlogloss:0.84500
[3]	validation_0-mlogloss:0.74575
[4]	validation_0-mlogloss:0.67671
[5]	validation_0-mlogloss:0.62040
[6]	validation_0-mlogloss:0.55434
[7]	validation_0-mlogloss:0.51041
[8]	validation_0-mlogloss:0.47192
[9]	validation_0-mlogloss:0.44875
[10]	validation_0-mlogloss:0.42948
[11]	validation_0-mlogloss:0.41780
[12]	validation_0-mlogloss:0.41072
[13]	validation_0-mlogloss:0.39857
[14]	validation_0-mlogloss:0.37938
[15]	validation_0-mlogloss:0.37635
[16]	validation_0-mlogloss:0.37091
[17]	validation_0-mlogloss:0.36098
[18]	validation_0-mlogloss:0.35443
[19]	validation_0-mlogloss:0.35284
[20]	validation_0-mlogloss:0.34643
[21]	validation_0-mlogloss:0.34709
[22]	validation_0-mlogloss:0.34573
[23]	validation_0-mlogloss:0.34520
[24]	validation_0-mlogloss:0.34519
[25]	validation_0-mlogloss:0.34382
[26]	validation_0-mlogloss:0.34629
[27]	validation_0-mlogloss:0.34486
[28]	validation_0-mlogloss:0.3

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=None, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [9]:
mean_Xtrain = X_train.mean()
std_Xtrain = X_train.std()

zscores_Xtrain = (X_train - mean_Xtrain)/std_Xtrain
zscores_Xtest = (X_test - mean_Xtrain)/std_Xtrain

In [10]:
clf1_xgb = xgb.XGBClassifier(objective='binary:logistic', num_class = 5, missing=None, seed=42)
clf1_xgb.fit(zscores_Xtrain, y_train, verbose=True, early_stopping_rounds=10, eval_metric='aucpr',
            eval_set=[(zscores_Xtest, y_test)])

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4], got [1. 2. 3. 4. 5.]

In [12]:
from sklearn.metrics import f1_score
import numpy as np

def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    f1 = f1_score(y_true, np.round(y_pred), average = 'weighted')
    return 'f1', f1

In [17]:
clf2_xgb = xgb.XGBClassifier(objective='binary:logistic', num_class = 5, missing=None, seed=42)
clf2_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric=f1_eval,
            eval_set=[(X_test, y_test)])

ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [8]:
y_one_hot = pd.DataFrame(np.random.randint(0, 2, size=(200, 5)), columns=[0, 1, 2, 3, 4])  # Example one-hot encoded y

# Convert one-hot encoded labels to integers
y33 = y_one_hot.idxmax(axis=1)
y33

0      0
1      0
2      1
3      1
4      2
      ..
195    1
196    0
197    1
198    1
199    0
Length: 200, dtype: int64

In [28]:
# Create the DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)

# Set the hyperparameters for XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': 5,
    'eval_metric': 'mlogloss'
}

# Train the XGBoost model
model = xgb.train(params, dtrain)

# Make predictions on new data
dtest = xgb.DMatrix([X_test,y_test])
predictions = model.predict(dtest)

print(predictions)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 45) + inhomogeneous part.

In [34]:
# Create an XGBClassifier object with multi-class objective
clf33_xgb = xgb.XGBClassifier(objective='multi:softmax', missing=None, random_state=42)

# Fit the classifier to the data
clf33_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric='f1_score', eval_set=[(X_test, y_test)])

XGBoostError: [11:43:03] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\metric\metric.cc:49: Unknown metric function f1_score

In [ ]:
print(sum(y_train)/len(y_train))
print(sum(y_train)/len(y_train))

In [ ]:
# using 10-fold Cross Validation technique and grid search
from sklearn.model_selection import KFold

# Define the number of folds
n_splits = 10

# Initialize the KFold object
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store the mean squared error for each fold
mse_list = []
Accuracy_list = []
F1_list = []
k_vlue = []
# Loop over the folds
for train_index, test_index in kf.split(X):
    
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Create an XGBClassifier object with multi-class objective
    clf_xgb = XGBClassifier(objective='multi:softmax', num_class=num_classes, random_state=42)

    # Fit the classifier to the data
    clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric={'mlogloss','f1_score','accuracy','auc'}
            eval_set=[(X_test, y_test)])
    